In [1]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
import sys
import dvats.utils as ut
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

import uni2ts
from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.plot import plot_next_multi
import pyarrow.feather as ft
from gluonts.transform.split import TFTInstanceSplitter
from gluonts.transform.sampler import TestSplitSampler
import numpy as np
import einops

In [2]:
from einops import rearrange

# Utilities

In [3]:
def get_sample_id(batch_size, seq_len):
    # Generamos un tensor que comienza en 0 y va incrementando de 1 en 1 hasta seq_len * batch_size - 1
    return torch.arange(batch_size * seq_len).view(batch_size, seq_len)
def get_variate_id(batch_size, seq_len, n_vars):
    total_repeats = seq_len // n_vars  # Cantidad de veces que cada variate_id debe repetirse para llenar seq_len
    remaining = seq_len % n_vars  # Resto que no completa una división exacta
    var_ids = torch.arange(n_vars).repeat_interleave(total_repeats)
    if remaining > 0:
        var_ids = torch.cat((var_ids, torch.arange(remaining)))
    return var_ids.unsqueeze(0).repeat(batch_size, 1)
    
def get_time_id(batch_size, seq_len, patches_per_var):
    patches_per_var = int (patches_per_var)
    # Creamos un tensor que repite una secuencia de 0 a patches_per_var-1
    time_ids = torch.arange(patches_per_var).repeat(seq_len // patches_per_var + 1)[:seq_len]
    # Repetimos este patrón para cada muestra en el batch
    return time_ids.unsqueeze(0).repeat(batch_size, 1)

# Main

In [4]:
shape = (521, 3, 30)
X = torch.rand(shape)

In [5]:
num_windows, num_variates, window_len = shape

In [6]:
batch_size = num_windows
seq_len = window_len * num_variates
#patch_size = 8
patch_size = 128
max_patch = patch_size
n_patches = int(np.ceil(seq_len / patch_size))

In [7]:
#target = rearrange(X, 'num_windows num_variates window_len -> num_windows window_len num_variates')

In [8]:
print(window_len)
print(patch_size)

30
128


In [9]:
import torch.nn.functional as F
# Calcular el número total de parches necesarios para una sola variable
n_patches_single_var = int(np.ceil(window_len / patch_size))
total_length_needed = n_patches_single_var * patch_size  # Longitud total requerida con padding

# Calcular el padding necesario si existe
padding_size = total_length_needed - window_len

# Añadir padding a X si es necesario
if padding_size > 0:
    X = F.pad(X, (0, padding_size), mode='constant', value=0)  # Pad al final de la última dimensión

# Ajustar n_patches considerando todas las variables
n_patches = n_patches_single_var * num_variates

In [10]:
y = rearrange(
    X,
    'batch_size n_vars (n_patches p) -> batch_size (n_vars n_patches) p',
    p = patch_size
)

In [11]:
y.shape

torch.Size([521, 3, 128])

In [12]:
#n_patches = int(np.ceil(window_len/patch_size))*num_variates
#print(n_patches)
#target = einops.rearrange(
#    X,
#    'batch_size window_len n_vars -> batch_size n_patches p',
#    p = patch_size, 
#    n_patches = n_patches
#)
#print(target.shape)

In [13]:
target = y
target.shape

torch.Size([521, 3, 128])

In [14]:
observed_mask = torch.ones_like(target, dtype = bool)
#prediction_mask = torch.zeros((target.shape[0], target.shape[1]), dtype = bool)
#observed_mask = torch.ones((batch_size, window_len, num_variates), dtype = bool)
#prediction_mask = torch.zeros((batch_size, window_len), dtype = bool)
prediction_mask = torch.zeros((batch_size, n_patches), dtype = bool)
sample_id = get_sample_id(batch_size, n_patches)
variate_id = get_variate_id(batch_size, n_patches, num_variates)

In [15]:
print(observed_mask.shape)
print(prediction_mask.shape)

torch.Size([521, 3, 128])
torch.Size([521, 3])


In [16]:
neg = ~prediction_mask.unsqueeze(-1)
neg.shape

torch.Size([521, 3, 1])

In [17]:
sol = observed_mask * neg
sol.shape

torch.Size([521, 3, 128])

In [18]:
print(target.shape)
print(observed_mask.shape)

torch.Size([521, 3, 128])
torch.Size([521, 3, 128])


In [19]:
einops.reduce(target * observed_mask, "... seq dim -> ... 1 seq", "sum").shape

torch.Size([521, 1, 3])

In [20]:
a = sample_id.unsqueeze(-1)
b = sample_id.unsqueeze(-2)
e = torch.eq(a,b)
print(a.shape, b.shape)

c = variate_id.unsqueeze(-1)
d = variate_id.unsqueeze(2)
f = torch.eq(c,d)

print(c.shape, d.shape)
id_mask = torch.logical_and(e,f)
print("id_mask ~", id_mask.shape)
reduced = einops.reduce(observed_mask, "... seq dim -> ... 1 seq", "sum")
print("Reduced: ", reduced.shape)
sol = id_mask * reduced
print(sol.shape)

torch.Size([521, 3, 1]) torch.Size([521, 1, 3])
torch.Size([521, 3, 1]) torch.Size([521, 3, 1])
id_mask ~ torch.Size([521, 3, 3])
Reduced:  torch.Size([521, 1, 3])
torch.Size([521, 3, 3])


In [21]:
#patch_size_tensor =  torch.zeros((batch_size, patch_size))+patch_size
patch_size_tensor =  torch.zeros((batch_size, n_patches, patch_size))+patch_size
patch_size_tensor.shape

torch.Size([521, 3, 128])

In [22]:
in_feat_size = target
feat_size = patch_size_tensor.float()

In [23]:
print(f"in_feat_size ~ {in_feat_size.shape}")
print(f"feat_size ~ {feat_size.shape}")
print(f"feat_size_sq ~ {feat_size.unsqueeze(-1).shape}")

in_feat_size ~ torch.Size([521, 3, 128])
feat_size ~ torch.Size([521, 3, 128])
feat_size_sq ~ torch.Size([521, 3, 128, 1])


In [24]:
torch.eq(in_feat_size, feat_size).unsqueeze(-1).shape

torch.Size([521, 3, 128, 1])

In [25]:
?? forecast_module.forward

Object ` forecast_module.forward` not found.


In [43]:
#sample_id = get_sample_id(batch_size, seq_len)
#variate_id = get_variate_id(batch_size, seq_len, num_variates)
#time_id = get_time_id(batch_size, seq_len, n_patches)
time_id = get_time_id(batch_size, seq_len, 1)
time_id.shape
#print(sample_id.shape)
#print(variate_id.shape)

torch.Size([521, 90])

In [27]:
target.shape

torch.Size([521, 3, 128])

In [44]:
forecast_module = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-small"),
    prediction_length=30, # No puede ser 0 porque falla el predictor... Pongo window len que es lo más parecido que tengo a "aprender mismos tamaños"
    context_length=30,
    patch_size=patch_size,
    num_samples=100,
    target_dim=3,
    feat_dynamic_real_dim = 0,
    past_feat_dynamic_real_dim = 0
)

In [45]:
input_transform = forecast_module.get_default_transform()
instance_splitter = TFTInstanceSplitter(
    instance_sampler=TestSplitSampler(),
    past_length=window_len,
    future_length=window_len, 
    observed_value_field="observed_target",
    time_series_fields=[],
    past_time_series_fields=[],
)

In [32]:
target.shape

torch.Size([521, 3, 128])

In [33]:
patch_size_tensor.shape

torch.Size([521, 3, 128])

In [46]:
res = forecast_module.module(
    target = target.float(),
    observed_mask = observed_mask.float(),
    sample_id = sample_id,
    time_id = time_id,
    variate_id = variate_id,
    prediction_mask = prediction_mask,
    patch_size = patch_size_tensor.float()
)

RuntimeError: The size of tensor a (128) must match the size of tensor b (3) at non-singleton dimension 2

In [38]:
print(f"Target ~ {target.shape}")
print(f"observed_mask ~ {observed_mask.shape}")
print(f"sample_id ~ {sample_id.shape}")
print(f"time_id ~ {time_id.shape}")
print(f"variate_id ~ {variate_id.shape}")
print(f"patch_size ~ {patch_size_tensor.shape}")

Target ~ torch.Size([521, 3, 128])
observed_mask ~ torch.Size([521, 3, 128])
sample_id ~ torch.Size([521, 3])
time_id ~ torch.Size([521, 90])
variate_id ~ torch.Size([521, 3])
patch_size ~ torch.Size([521, 3, 128])


In [ ]:
forecast_module.module.named_modules

In [ ]:
print("observed: ", observed_mask.shape)
print("prediction: ", prediction_mask.shape)

In [ ]:
?? forecast_module.module.forward